In [5]:
# RAG
# 우리회사의 복지제도는?
# LLM은 학습데이터에 없는 최신/특정 정보를 모름

# RAG 해결책  (Tetrieval Augmented Generation) = 검색 + 생성
# 1. 회사 문서에서 관련 정보 검색
# 2. 검색된 정보를 LLM에게 컨텍스트로 제공
# 3. llm이 컨텍스 기반으로 정확한 답변 생성

# [준비 단계]
# 문서들 ->청크분할->벡터변환->벡터DB저장

#[쿼리 단계]
# 질문->벡터변환->유사도검색->상위 k개 선택 -> 컨텍스트 +질문->LLM->답변

In [6]:
%pip install llama-index

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from llama_index.core import Document, VectorStoreIndex

In [9]:
# from google.colab import userdata
# userdata.get('OPENAI_API_KEY')[:5]

In [11]:
# 1 문서 준비
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
document = [
    Document(text='대한민국의 수도는 서울입니다.'),
    Document(text="프랑스의 수도는 파리 입니다.")
]
# 2 인덱스 생성(자동으로 벡터화)
# 각 청크를 openai api 로 벡터화
# 인메모리방식으로 벡터 스토어에 저장
index = VectorStoreIndex.from_documents(document)

# 3 쿼리 엔진 생성
query_engine = index.as_query_engine(similarity_top_k=1)

# 4 쿼리 실행
response = query_engine.query("대한민국의 수도는 어디입니까?")
print(response)

서울


In [12]:
# 청크 : 문서검색의 최소단위 모델이 한번에 처리할수 있는 길이로 잘라낸 텍스트
# 모델 입력 길이 제한, 문서가 길면 한번에 처리 할 수 없어서 청크로 나눠 처리
# 벡터 DB에서 문서전체가 아니라 청크단위로 벡터화
# 질문과 유사한 작은 단위를 찾아 답변을 생성
# 전체문서를 이해하는 대신 청크별로 처리해서  중요한 부분에 집중

# 작을수록 : 정확한 검색, 많은 api 호출
# 클수록 : 넓은 컨텍스트, 적은 api 호출
from llama_index.core import Settings
Settings.chunk_size = 512  # 기본값
# Settings.chunk_overlap = 128  # 기본값
Settings.chunk_overlap = 50 # 청크 간 겹침

# 유사도 임계값 설정
from llama_index.core.postprocessor import SimilarityPostprocessor
query_engine = index.as_query_engine(
    similarity_top_k=2,  # 유사도 상위 2
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)  # 유사도 0.7미만의 문서는 제외 (노이즈 제거)
    ]
)
# 배치 처리
Settings.embed_batch_size = 100

한국어 데이터로 RAG 구현

In [13]:
# 한국어 데이터로 RAG 구현
from llama_index.core import Document,VectorStoreIndex,Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

In [14]:
#. 1. LLM, 임베딩 모델 설정
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [15]:
# 2. 문서 준비
documents = [
    Document(
        text="김치는 한국의 대표적인 발효 음식입니다. 배추에 고춧가루, 마늘, 생강 등을 넣어 만듭니다.",
        metadata={"source": "한국 음식 백과", "category": "반찬"}
    ),
    Document(
        text="비빔밥은 밥 위에 여러 가지 나물과 고기, 계란을 올려 고추장과 섞어 먹는 음식입니다.",
        metadata={"source": "한국 음식 백과", "category": "밥 요리"}
    ),
    Document(
        text="불고기는 양념한 소고기를 구워 먹는 한국의 전통 음식입니다. 달콤하고 짭짤한 맛이 특징입니다.",
        metadata={"source": "한국 음식 백과", "category": "고기 요리"}
    ),
    Document(
        text="떡볶이는 가래떡에 고추장 양념을 넣어 볶은 한국의 길거리 음식입니다. 달콤하고 매운 맛이 특징입니다.",
        metadata={"source": "한국 음식 백과", "category": "분식"}
    ),
]

In [16]:
# 3. 벡터 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

In [17]:
# 4. 쿼리 엔진 생성
query_engine = index.as_query_engine(
    similarity_top_k=2,
    # node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
)

In [18]:
# 5. 질문하기
questions = [
    '김치는 어떤 음식인가요?',
    '비빔밥을 어떻게 먹나요?',
    '한국의 고기 요리에는 뭐가 있나요?'
]
for q in questions:
  response = query_engine.query(q)
  print(f'질문:{q} 답변 :{response}')

질문:김치는 어떤 음식인가요? 답변 :김치는 한국의 대표적인 발효 음식으로, 배추에 고춧가루, 마늘, 생강 등을 넣어 만들어집니다.
질문:비빔밥을 어떻게 먹나요? 답변 :비빔밥은 밥 위에 여러 가지 나물과 고기, 계란을 올린 후 고추장과 섞어 먹습니다.
질문:한국의 고기 요리에는 뭐가 있나요? 답변 :한국의 고기 요리 중 하나로 불고기가 있습니다. 불고기는 양념한 소고기를 구워 먹는 전통 음식으로, 달콤하고 짭짤한 맛이 특징입니다.


LLM 캐시

In [20]:
# 동일한 질문을 반복하면
from openai import OpenAI
import openai
import time
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
question = '대한민국의 수도는'

start = time.time()
for i in range(100):
  response = client.chat.completions.create(
      model = 'gpt-4o-mini',
      messages = [{'role':'user', 'content':question}],
      temperature = 0
  )
  answer = response.choices[0].message.content
elapsed_time = time.time() - start
print(f'elapsed_time:{elapsed_time}')


elapsed_time:81.56446552276611


In [21]:
# 캐시.. 완전 일치 캐시(Exact Match Cache)
# 동일한 입력 ->저장된 응답 반환
# 장점 : 구현이 간단하고, 100% 정확
# 단점 : 완전히 같아야만 작동

# 대한민국의 수도는?  캐시 히트
# 대한민국 수도는?    캐시 미스(다른 문자열)
# 한국의 수도는?      캐시 미스

cache = {}
response = []
start = time.time()
for i in range(100):
  if question in cache:
    asnwer = cache[question]
  else:
    response = client.chat.completions.create(
      model = 'gpt-4o-mini',
      messages = [{'role':'user', 'content':question}],
      temperature = 0
    )
    answer = response.choices[0].message.content
    cache[question] = answer
elapsed_time = time.time() - start
print(f'elapsed time: {elapsed_time}')

elapsed time: 0.6113805770874023


In [22]:
# 의미적 캐시 (Semantic Cache)
# 으미가 비슷한 입력->저장된 응답 반환
# 1. 유사한 프롬프트 검색
# 2. 유사도 확인  (특정 임계값을 지정해서 그 값에따라서 답변 채택 종료)
# 3. 비슷한게 없으면 llm 호출

# 장점 :
# 높은 히트율
# 다양한 표현허용 유연함
# 비용 절감

# 단점
# 약간 느림
# 벡터 DB 필요

In [23]:
%pip install chromadb

  Using cached onnxruntime-1.23.2-cp313-cp313-win_amd64.whl.metadata (5.3 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/21.7 MB ? eta -:--:--
   ----- ---------------------------------- 3.1/21.7 MB 27.8 MB/s eta 0:00:01
   --------- ------------------------------ 5.2/21.7 MB 12.6 MB/s eta 0:00:02
   -------------------- ------------------- 11.0/21.7 MB 18.1 MB/s eta 0:00:01
   --------------------------------- ------ 18.1/21.7 MB 22.4 MB/s eta 0:00:01
   ---------------------------------------- 21.7/21.7 MB 22.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 29.3 MB/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.35.0 requires typing_extensions~=4.14.0, but you have typing-extensions 4.15.0 which is incompatible.
selenium 4.35.0 requires urllib3[socks]<3.0,>=2.5.0, but you have urllib3 2.3.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
# 벡터스토어  -> DB
# 문서나 텍스트를 벡터로 변환한후에 저장 -> 유사도 기반 검색 기능
import chromadb
# 클라이언트 생성
client = chromadb.Client()
# 컬렉션 생성
# collection = client.create_collection("my_collection2")
# 문서와 임베딩 준비
texts = [
    '대한민국의 수도는 서울입니다.',
    '프랑스의 수도는 파리 입니다.',
    '서울은 한국의 정치,경제 중심지 입니다.'
]
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts).tolist()
embeddings  # (3,384)


c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\31799\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[[-0.023814842104911804,
  0.08707407861948013,
  0.07979423552751541,
  -0.010389544069766998,
  -0.03870178014039993,
  -0.02934100292623043,
  0.10985817760229111,
  0.015440710820257664,
  -0.011003022082149982,
  -0.009104174561798573,
  0.08716966956853867,
  -0.04601134732365608,
  0.0490763820707798,
  -0.07741817086935043,
  0.08031683415174484,
  -0.03652264177799225,
  0.044351544231176376,
  0.05164162814617157,
  -0.10573045164346695,
  0.031718332320451736,
  0.02549370564520359,
  -0.0049087596125900745,
  0.02975667640566826,
  0.028301825746893883,
  -0.06878553330898285,
  -0.0273138340562582,
  0.005474119447171688,
  -0.012014391832053661,
  0.07081886380910873,
  0.04117822274565697,
  -0.02017916552722454,
  0.06898639351129532,
  0.028437959030270576,
  0.07568581402301788,
  -0.06454992294311523,
  0.044827889651060104,
  -0.062241293489933014,
  0.019267575815320015,
  -0.01746961660683155,
  -0.008292272686958313,
  -0.15329048037528992,
  -0.1490267813205719,

In [29]:
# 문서추가
ids = ['doc1','doc2','doc3']
collection.add(
    ids=ids,
    documents=texts,
    embeddings=embeddings
)

In [30]:
# 유사도 검색
query = '조선의 수도는 어디인가요?'
query_embedding = model.encode([query]).tolist()
results = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)
print(results)

{'ids': [['doc1']], 'embeddings': None, 'documents': [['대한민국의 수도는 서울입니다.']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[0.3832094967365265]]}


In [31]:
# 다층 캐시 전략
# 메모리(완전일치) - 미스 벡터DB(의미적) - 미스 LLM호출

     

In [32]:
# 완전 일치
class SimpleCache:
  def __init__(self):
    self.cache = {}  # 딕셔너리
    self.hits = 0
    self.misses = 0
  def get(self, key):
    if key in self.cache:
      self.hits += 1
      return self.cache[key]
    self.misses += 1
    return None
  def set(self,key,value):
    self.cache[key] = value
  def state(self):
    total = self.hits + self.misses
    hit_rate = self.hits / total*100 if total > 0 else 0
    return{
        'hits':self.hits,
        'misses':self.misses,
        'hit_rate':hit_rate
    }

In [33]:
import openai
from openai import OpenAI
# openai.api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))
def call_llm(question):
  response = client.chat.completions.create(
      model = 'gpt-4o-mini',
      messages = [{'role':'user', 'content':question}],
      temperature = 0
    )
  return response.choices[0].message.content

In [34]:
cache = SimpleCache()
questions = [
    '대한민국의 수도는?',
    '대한민국의 수도는?',  # 캐시 히트
    '한국의 수도는?',      # 캐시 미스(다른 문자열)
]
for q in questions:
  cached = cache.get(q)
  if cached:
    print(f'캐시 : {cached}')
  else:
    response = call_llm(q)
    cache.set(q,response)
    print(f' llm : {response}')
print(cache.state())

 llm : 대한민국의 수도는 서울입니다.
캐시 : 대한민국의 수도는 서울입니다.
 llm : 한국의 수도는 서울입니다.
{'hits': 1, 'misses': 2, 'hit_rate': 33.33333333333333}


In [36]:
# 2. 의미적 유사성 - 벡터 DB chromadb
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
class SemanticCache:
    def __init__(self):
        self.client = chromadb.Client()
        self.embedding_function = OpenAIEmbeddingFunction(
            api_key = os.getenv('OPENAI_API_KEY'),
            model_name = 'text-embedding-3-small'
        )
        self.collection = self.client.create_collection(
            name='semantic_cache',
            embedding_function=self.embedding_function,
            metadata={'hnsw:space':'cosine'}
        )
    def get(self, query, threshold=0.15):
        results = self.collection.query(
            query_texts=[query],
            n_results=1
        )
        if results['distances'][0] and results['distances'][0][0] < threshold:
            return results['metadatas'][0][0]
        return None
    def set(self, query, response):
        import uuid
        self.collection.add(
            documents=[query],
            metadatas=[{'response': response}],
            ids = [str(uuid.uuid4())]
        )

In [40]:
# SemanticCache 사용 예제
# cache = SemanticCache()
questions = [
    '대한민국의 수도는?',
    '대한민국의 수도는?',  # 캐시 히트
    '한국의 수도는?',      # 캐시 미스(다른 문자열)
]
for q in questions:
    cached = cache.get(q)
    if cached:
        print(f'HIT : {q} - {cached['response']}')
    else:
        response = call_llm(q)
        cache.set(q,response)
        print(f'llm : {response}')

HIT : 대한민국의 수도는? - 대한민국의 수도는 서울입니다.
HIT : 대한민국의 수도는? - 대한민국의 수도는 서울입니다.
HIT : 한국의 수도는? - 한국의 수도는 서울입니다.


In [43]:
'''
L1 메모리 내부메모리..dictionary
L2 메모리 벡터DB - 의미적 유사성
L3 메모리 LLM호출
'''
class MulltiLevelCache:
  def __init__(self) -> None:
    self.l1_cach = SimpleCache()  # 메모리방식 dictionary   완전일치
    self.l2_cach = SemanticCache() # ChoromaDB 벡터DB  유사도방식
  def stats(self):
    print(f'L1 catch: {self.l1_cach.cache}')
  def get(self,key):
    cached = self.l1_cach.get(key)
    if cached:
      print('L1 cache')
      return cached
    cached = self.l2_cach.get(key)
    if cached:
      print('L2 cache')
      self.l1_cach.set(key,cached)
      return cached
    # LLM 호출
    print('LLM')
    response = call_llm(key)
    self.l1_cach.set(key,response)
    self.l2_cach.set(key,response)
    return response
mulltiLevelCache = MulltiLevelCache()

InternalError: Collection [semantic_cache] already exists

In [42]:
mulltiLevelCache.get('america의  수도는')

NameError: name 'mulltiLevelCache' is not defined